### Import Libraries 

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pandas as pd
import numpy as np

### Model Implementation

In [ ]:
df = pd.read_csv('datasets/preprocessed_train.csv')

# Drop rows with NaN values in the 'preprocessed_review' column
df = df.dropna(subset=['preprocessed_review'])

# Convert ratings to categorical labels
labels = to_categorical(df['rating'] + 1)  # Adding 1 to convert -1, 0, 1 to 0, 1, 2

# Tokenize the preprocessed reviews
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['preprocessed_review'])
sequences = tokenizer.texts_to_sequences(df['preprocessed_review'])
max_sequence_length = 100  # Set your desired sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
# Define the BiLSTM model
vocab_size = len(tokenizer.word_index) + 1  # Vocabulary size

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_sequence_length))
model.add(Bidirectional(LSTM(units=64, return_sequences=True))) 
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=32)))  
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))  # Output layer with 3 classes

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
model.summary()
model.fit(padded_sequences, labels, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(padded_sequences, labels, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

### Test Prediction

In [ ]:
test_data = pd.read_csv('datasets/preprocessed_test.csv')

# Tokenize the preprocessed reviews in the test data
sequences_test = tokenizer.texts_to_sequences(test_data['preprocessed_review'])
padded_sequences_test = pad_sequences(sequences_test, maxlen=padded_sequences.shape[1])

In [ ]:
predictions = model.predict(padded_sequences_test)
predicted_ratings = np.argmax(predictions, axis=1) - 1

In [ ]:
test_data['rating'] = predicted_ratings
test_data.to_csv('predicted_test.csv', index=False)